In [2]:
import pandas as pd
import numpy as np
from kerasSentModel import kerasSentiment


data_df = pd.read_csv( "labelledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )

Using TensorFlow backend.


Let us look at the data


In [3]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
id           25000 non-null object
sentiment    25000 non-null int64
review       25000 non-null object
dtypes: int64(1), object(2)
memory usage: 586.0+ KB


In [4]:
data_df.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


Extract sentiment count

In [5]:
data_df.sentiment.value_counts()

1    12500
0    12500
Name: sentiment, dtype: int64

We shall select a subset for training and holdout the rest for validation

In [12]:
train_pct = 0.8
train_samples = np.int(train_pct * train_df.shape[0])
train_df = data_df.iloc[0:train_samples,:]


In [15]:
#replace empty cells with Nan
train_df.review.replace('', np.nan, inplace=True)
#drop null rows
train_df.dropna(inplace=True)

C:\Users\takalyan\Documents\Anaconda\lib\site-packages\pandas\core\generic.py:5886: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
C:\Users\takalyan\Documents\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [16]:
#Instantiate model
kerasModel = kerasSentiment()

Cleanup review column to remove HTML markup and non text characters

In [17]:
train_df['review'] = train_df.review.apply(lambda x : kerasModel.cleanup_review(x))

C:\Users\takalyan\Documents\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [18]:
#assign train dataframe and define which columns have the data
kerasModel.text_df = train_df
kerasModel.text_colname = "review"
kerasModel.num_classes = 2
kerasModel.sentiment_colname = "sentiment"

In [19]:
#Get padded docs
padded_docs, labels= kerasModel.encodeTrainText()

Encoding and padding text..

Max sentence length in doc:2487

Encoding text..

Padding text..



In [20]:
#Create the model
#kerasModel.createModel(embedding_matrix=embedding_matrix,useTrainedEmbedding=True)
kerasModel.createModel()

Creating Keras model..

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 2487, 100)         5557300   
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                30200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 51        
Total params: 5,587,551
Trainable params: 30,251
Non-trainable params: 5,557,300
_________________________________________________________________
None


In [21]:
#Model hyperparameters
kerasModel.lstm_units = 10
kerasModel.early_stop_epochs = 10
kerasModel.epochs = 5
kerasModel.batch_size = 128
kerasModel.verbose = True



In [ ]:
#Train the model
kerasModel.trainModelKfold(padded_docs,labels)


Training model using K-Fold cross val..



In [ ]:
#Save model 
kerasModel.saveModel('kerasModelLSTM')